| <h1> Data Science Coursera Capstone Project <h1/> |
:---:

This project is for the IBM Data Science course offered on Coursera. This Capstone project will be analyzing neighborhoods in Toronto Canada to determine where to move. Location data and Machine Learning will be used to come to the best conclusion for the situation. 

In [3]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Course!")

Hello Capstone Course!
